### Imports and loads

In [1]:
import argparse
import cv2
import itertools
import numpy
import os
import pickle
from sklearn.decomposition import TruncatedSVD

import parameters
from cnn_feature_extraction import extract_features_cnn
from finder import return_similar
from detect_objects import crop_box_for_class
from app import search_engine
from app.web_interface import get_engine, get_directories

QUERY_IMAGES = './app/static/images/room_scenes'
OBJECT_FEATURES_FILE = 'pickles/object_features.pickle'

Using Theano backend.


Gallery yolo detections loaded!
Gallery CNN features loaded!
Model extension is resnet
<_io.BufferedReader name='pickles/clock_resnet.pickle'> loaded
<_io.BufferedReader name='pickles/bed_resnet.pickle'> loaded
<_io.BufferedReader name='pickles/chair_resnet.pickle'> loaded
<_io.BufferedReader name='pickles/plant_pot_resnet.pickle'> loaded
<_io.BufferedReader name='pickles/sofa_resnet.pickle'> loaded
<_io.BufferedReader name='pickles/table_resnet.pickle'> loaded
All engines initiated
2.1343648433685303 has passed since start for loading pickles


In [2]:
# load similarity
with open('pickles/similarity.p', 'rb') as f:
    similarity = pickle.load(f)
def similarity_for_pair(a,b):
    return similarity[a][b]

In [23]:
#load image features reduced by svd
with open('pickles/images_features_reduced.p', 'rb') as f:
    images_features_25 = pickle.load(f)

In [22]:
TEXT_QUERIES = ['comfortable',
'child',
'light',
'chair',
'fabric',
'soft',
'cushion',
'plant pot',
'bookcase',
'box',
'white',
'natural',
'playful',
'relaxing',
'colours',
'children',
'watercolour',
'space',
'organized',
'adult',
'extendable',
'basic',
'desired',
'yellow',
'cosy',
'comforting',
'loft',
'junior',
'texture',
'suitable',
'decorative',
'textile',
'black',
'printed',
'colourful',
'patterns',
'creative',
'safe',
'handmade',
'unique',
'dining',
'social',
'smooth']

### Auxiliary functions

In [5]:
def extract_and_save_features(image_path):
    """Working on Theano"""
    try:
        with open(OBJECT_FEATURES_FILE, 'rb') as handle:
            object_features = pickle.load(handle)
    except FileNotFoundError:
        print('No object features file found! Creating a new one')
        object_features = {}
    if image_path in object_features:
        return(object_features[image_path])
    else:
        print('Extracting features for', image_path)
        extracted_features = extract_features_cnn(image_path)
        object_features[image_path] = extracted_features
        fileObject = open(OBJECT_FEATURES_FILE, 'wb')
        pickle.dump(object_features, fileObject)
        fileObject.close()
        return extracted_features

In [6]:
def similarity_for_list(list_of_images, similarity_file='pickles/similarity.p'):
    total_sum = 0
    pairs = 0
    for pair in itertools.combinations(list_of_images, 2):
        try:
            add = similarity_for_pair(*pair)
            total_sum += add
            pairs +=1
        except KeyError:
            print(pair, 'Not in the dictionary')
    print('Total similarity for list:', total_sum/pairs)
    return total_sum/pairs

### Blender functions

In [7]:
def return_base_blender_results(image_path, text_query):
    """Searches for visually similar, word2vec and countvect results to object in picture"""
    search_dir, engine, static_path, object_class, bound_boxes = get_engine(image_path, parameters.ACCURACY_CLASSES)
    object_image = crop_box_for_class(bound_boxes, image_path, 'app/static/bounding_boxes', object_class)
    if text_query == "":
        text_query = object_class
    similar_images = [os.path.basename(image) for image in return_similar(object_image, search_dir, engine, parameters.NB_MATCHES)]
    images_countvect = [os.path.basename(image) for image in search_engine.process_query(text_query)]
    images_w2vec = [os.path.basename(image) for image in search_engine.process_query_w2vec(text_query)]
    top4_blend = similar_images[:4] + images_w2vec[:4] + images_countvect[:4]
    return top4_blend, object_class

In [8]:
def return_text_results(text_query):
    images_w2vec = [os.path.basename(image) for image in search_engine.process_query_w2vec(text_query)]
    return images_w2vec

In [9]:
def return_feature_blender_results(image_path, text_query):
    """Looks for images from initial query sharing the same deep features"""
    search_dir, engine, static_path, object_class, bound_boxes = get_engine(image_path, parameters.ACCURACY_CLASSES)
    object_image = crop_box_for_class(bound_boxes, image_path, 'app/static/bounding_boxes', object_class)
    if text_query == "":
        text_query = object_class
    if object_class == "pottedplant":
        images_path = 'app/static/images/plant_pot'
    elif object_class == "diningtable":
        images_path = 'app/static/images/table'
    elif object_class == "all":
        return [], "all"
    else:
        images_path = 'app/static/images/' + object_class        
    similar_images = [os.path.join(images_path, os.path.basename(image)) for image in return_similar(object_image, search_dir, engine, parameters.NB_MATCHES)]
    images_countvect = ['app' + image for image in search_engine.process_query(text_query)]
    images_w2vec = ['app' + image for image in search_engine.process_query_w2vec(text_query)]
    initial_query = similar_images + images_countvect + images_w2vec
    initial_image = initial_query[0]
    initial_query.remove(initial_image)    
    distances = {}
    initial_features = extract_and_save_features(initial_image)
    for product_path in initial_query:
        image_features = extract_and_save_features(product_path)
        distances[product_path] = numpy.linalg.norm(image_features-initial_features)
    sorted_array = [key for key in sorted(distances, key=distances.__getitem__)]
    return [os.path.basename(filename) for filename in sorted_array[:12]], object_class
    

In [47]:
def return_svd25_blender_results(image_path, text_query):
    search_dir, engine, static_path, object_class, bound_boxes = get_engine(image_path, parameters.ACCURACY_CLASSES)
    object_image = crop_box_for_class(bound_boxes, image_path, 'app/static/bounding_boxes', object_class)
    if text_query == "":
        text_query = object_class
    images_w2vec = [ os.path.basename(image) for image in search_engine.process_query_w2vec(text_query)]
    image_path_key = 'images/room_scenes/' + os.path.basename(image_path)
    image_features_reshaped = images_features_25[image_path_key]
    images_svd25 = [os.path.basename(image) for image in search_engine.process_features_query(image_features_reshaped)]
    print('svd25 result images', images_svd25 + images_w2vec)
    return(images_svd25 + images_w2vec), object_class

### Testing functions

In [11]:
res, _ = return_base_blender_results('app/static/images/room_scenes/ikea-get-the-trend-artisan__1364321210884-s4.jpg', "")


ikea-get-the-trend-artisan__1364321210884-s4.jpg is already in detections file!
Nothing to copy - no yolo predictions image found
Detected object class from bounding boxes pottedplant
Getting directories for  pottedplant
Searching for results in  ./app/static/images/plant_pot
Return_similar for app/static/bounding_boxes/pottedplant_ikea-get-the-trend-artisan__1364321210884-s4.jpg


In [29]:
print(images_features_25['images/room_scenes/ikea-get-the-trend-artisan__1364321210884-s4.jpg'])

[ 0.01992422 -0.01054213  0.00383847  0.01689113  0.00233677  0.00127325
  0.00287853 -0.00088592 -0.00364683 -0.00358941 -0.00281358 -0.00168951
 -0.00050659 -0.00126251  0.00033831 -0.00227791  0.00146312  0.00051476
  0.00315344  0.00351194  0.00040293  0.00048094 -0.00144346 -0.0005173
 -0.00201333]


In [43]:
similarity['503.214.28.jpg']['503.214.28.jpg']

1.0

In [13]:
similarity_for_list(res)

Total similarity for list: 0.112638338376735


0.112638338376735

In [36]:
return_svd25_blender_results('app/static/images/room_scenes/ikea-get-the-trend-artisan__1364321210884-s4.jpg', "cozy")



['app/static/images/102.883.03.jpg',
 'app/static/images/503.214.28.jpg',
 'app/static/images/302.581.21.jpg',
 'app/static/images/202.244.19.jpg',
 'app/static/images/303.541.08.jpg',
 'app/static/images/702.729.12.jpg',
 'app/static/images/500.210.76.jpg',
 'app/static/images/bed/802.538.09.jpg',
 'app/static/images/802.962.48.jpg',
 'app/static/images/502.962.97.jpg',
 'app/static/images/203.086.97.jpg',
 'app/static/images/302.980.23.jpg']

## Calculate similarity for vanilla Visual Search

In [1]:
def similarity_visual_search(images_path, text_query):
    similarities = []
    for image_name in os.listdir(images_path):
        image_path = os.path.join(images_path,image_name)
        if cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) is not None:
            search_dir, engine, static_path, object_class, bound_boxes = get_engine(image_path, parameters.ACCURACY_CLASSES)
            object_image = crop_box_for_class(bound_boxes, image_path, 'app/static/bounding_boxes', object_class)
            if object_class == "all":
                continue
            else:
                similar_images = [os.path.basename(image) for image in return_similar(object_image, search_dir, engine, 12)]
                results_similarity = similarity_for_list(similar_images)
                similarities.append(results_similarity)
    print('Mean similarity for set', numpy.mean(similarities))


In [ ]:
# similarity_visual_search(QUERY_IMAGES, "")

## Calculate similarity for Word 2 Vec

In [12]:
def similarity_word2vec(text_query):
    text_results = return_text_results(text_query)
    results_similarity = similarity_for_list(text_results)
    return results_similarity
    

In [15]:
similarities = []
for query in TEXT_QUERIES:   
    sim_result = similarity_word2vec(query)
    similarities.append((query, sim_result))
print(similarities)

Total similarity for list: 0.30189092045632127
Similarity is comfortable 0.30189092045632127
Total similarity for list: 0.024035005469604628
Similarity is child 0.024035005469604628
Total similarity for list: 0.2793248945147679
Similarity is light 0.2793248945147679
Total similarity for list: 0.31979996874511646
Similarity is chair 0.31979996874511646
Total similarity for list: 0.03706829192061259
Similarity is fabric 0.03706829192061259
Total similarity for list: 0.04222534771057979
Similarity is soft 0.04222534771057979
Total similarity for list: 0.07898109079543682
Similarity is cushion 0.07898109079543682
Total similarity for list: 0.00212533208313799
Similarity is plant pot 0.00212533208313799
Total similarity for list: 0.24194405375839978
Similarity is bookcase 0.24194405375839978
Total similarity for list: 0.18962337865291456
Similarity is box 0.18962337865291456
Total similarity for list: 0.2036255664947648
Similarity is white 0.2036255664947648
Total similarity for list: 0.029

## Calculate similarity for Base Blender

In [9]:
def similarity_base_blender(images_path, text_query):
    similarities = []
    for image_name in os.listdir(images_path):
        image_path = os.path.join(images_path,image_name)
        if cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) is not None:
            blender_results, object_class = return_base_blender_results(image_path, text_query)
            if object_class == "all":
                continue
            else:
                results_similarity = similarity_for_list(blender_results)
                similarities.append(results_similarity)
    print('Mean similarity for set', numpy.mean(similarities))
    return numpy.mean(similarities)

In [2]:
# similarity_base_blender(QUERY_IMAGES, "watercolour")

In [36]:
base_similarities = []
for text_qr in ['space', 'organized', 'adult']:
    base_similarity= similarity_base_blender(QUERY_IMAGES, text_qr)
    base_similarities.append((text_qr, base_similarity))

Detected object class from bounding boxes chair
Return_similar for app/static/bounding_boxes/chair_ikea-a-bed-that-folds-away-to-be-a-sofa-by-day__1364309472525-s4.jpg
Total similarity for list: 0.37065450567560265
Detected object class from bounding boxes chair
Return_similar for app/static/bounding_boxes/chair_ikea-a-bright-kitchen-idea-with-custom-character__1364316015597-s4.jpg
Total similarity for list: 0.24425691514299114
Detected object class from bounding boxes pottedplant
Return_similar for ./app/static/images/room_scenes/ikea-a-clean-way-to-display-or-tidy-away.__1364309466684-s4.jpg
Total similarity for list: 0.22647714841807673
Detected object class from bounding boxes chair
Return_similar for app/static/bounding_boxes/chair_ikea-a-colourful-room-for-kids-to-get-creative__1364308442737-s4.jpg
Total similarity for list: 0.3510917899103552
Detected object class from bounding boxes chair
Return_similar for app/static/bounding_boxes/chair_ikea-a-contemporary-kitchen-for-big-che

In [37]:
print(base_similarities)

[('space', 0.24935880514688191), ('organized', 0.12337951219467488), ('adult', 0.079991879229663373)]


## Calculate similarity for Feature Blender

In [15]:
def similarity_feature_blender(images_path, text_query):
    similarities = []
    for image_name in os.listdir(images_path):
        image_path = os.path.join(images_path,image_name)
        if cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) is not None:
            blender_results, object_class = return_feature_blender_results(image_path, text_query)
            if object_class == "all":
                continue
            else:
                results_similarity = similarity_for_list(blender_results)
                similarities.append(results_similarity)
    print('Mean similarity for set', numpy.mean(similarities))
    return numpy.mean(similarities)
    

In [8]:
my_res = similarity_feature_blender(QUERY_IMAGES, "decorative")

Detected object class from bounding boxes chair
Getting directories for  chair
Searching for results in  ./app/static/images/chair
Return_similar for app/static/bounding_boxes/chair_ikea-a-bed-that-folds-away-to-be-a-sofa-by-day__1364309472525-s4.jpg
return_similar was completed in 0.02 sec
Extracting features for app/static/images/902.387.38.jpg
Using model resnet
reshaping resnet
extract_features_cnn was completed in 19.90 sec
Extracting features for app/static/images/101.841.74.jpg
Using model resnet
reshaping resnet
extract_features_cnn was completed in 0.26 sec
Extracting features for app/static/images/802.600.94.jpg
Using model resnet
reshaping resnet
extract_features_cnn was completed in 0.25 sec
Extracting features for app/static/images/590.272.67.jpg
Using model resnet
reshaping resnet
extract_features_cnn was completed in 0.27 sec
Extracting features for app/static/images/701.550.17.jpg
Using model resnet
reshaping resnet
extract_features_cnn was completed in 0.31 sec
Extract

In [9]:
print (my_res)

0.251665829155


In [34]:
feature_similarities = []
for text_qr in TEXT_QUERIES[7:8]:
    feature_similarity= similarity_feature_blender(QUERY_IMAGES, text_qr)
    feature_similarities.append((text_qr, feature_similarity))

Detected object class from bounding boxes chair
Return_similar for app/static/bounding_boxes/chair_ikea-a-bed-that-folds-away-to-be-a-sofa-by-day__1364309472525-s4.jpg
Extracting features for app/static/images/plant_pot/702.858.58.jpg
Using model resnet
reshaping resnet
extract_features_cnn was completed in 0.27 sec
Extracting features for app/static/images/plant_pot/701.866.55.jpg
Using model resnet
reshaping resnet
extract_features_cnn was completed in 0.33 sec
Extracting features for app/static/images/102.347.82.jpg
Using model resnet
reshaping resnet
extract_features_cnn was completed in 0.34 sec
Extracting features for app/static/images/902.691.50.jpg
Using model resnet
reshaping resnet
extract_features_cnn was completed in 0.33 sec
Extracting features for app/static/images/301.316.55.jpg
Using model resnet
reshaping resnet
extract_features_cnn was completed in 0.33 sec
Extracting features for app/static/images/001.924.24.jpg
Using model resnet
reshaping resnet
extract_features_cn

In [35]:
print(feature_similarities)

[('plant pot', 0.15953985433024501)]


## Calculate similarity for SVD blender

In [48]:
def similarity_svd25_blender(images_path, text_query):
    similarities = []
    for image_name in os.listdir(images_path):
        image_path = os.path.join(images_path,image_name)
        if cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) is not None:
            blender_results, object_class = return_svd25_blender_results(image_path, text_query)
            if object_class == "all":
                continue
            else:
                results_similarity = similarity_for_list(blender_results)
                similarities.append(results_similarity)
    print('Mean similarity for set', numpy.mean(similarities))
    return numpy.mean(similarities)

In [ ]:
my_res = similarity_svd25_blender(QUERY_IMAGES, "social")

ikea-a-bed-that-folds-away-to-be-a-sofa-by-day__1364309472525-s4.jpg is already in detections file!
Nothing to copy - no yolo predictions image found
Detected object class from bounding boxes chair
Getting directories for  chair
Searching for results in  ./app/static/images/chair
svd25 result images ['503.214.28.jpg', '802.644.31.jpg', '503.118.15.jpg', '303.541.08.jpg', '202.244.19.jpg', '702.729.12.jpg', '800.992.24.jpg', '001.414.01.jpg', '202.410.08.jpg', '301.933.23.jpg', '902.824.39.jpg', '201.840.41.jpg']
Total similarity for list: 0.2145363622156871
ikea-a-bright-kitchen-idea-with-custom-character__1364316015597-s4.jpg is already in detections file!
Nothing to copy - no yolo predictions image found
Detected object class from bounding boxes chair
Getting directories for  chair
Searching for results in  ./app/static/images/chair
svd25 result images ['402.520.10.jpg', '202.247.73.jpg', '201.514.08.jpg', '702.729.12.jpg', '302.865.48.jpg', '303.541.08.jpg', '800.992.24.jpg', '001.4

In [ ]:
print(my_res)